In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile8 = pd.read_pickle("BBB_Train_Decile8.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile8.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile8, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 8')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3206 - acc: 0.8834 - val_loss: 0.2965 - val_acc: 0.8873
Epoch 39/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3220 - acc: 0.8840 - val_loss: 0.2967 - val_acc: 0.8860
Epoch 40/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3169 - acc: 0.8869 - val_loss: 0.2948 - val_acc: 0.8873
Epoch 41/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3183 - acc: 0.8876 - val_loss: 0.2957 - val_acc: 0.8847
Epoch 42/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3127 - acc: 0.8866 - val_loss: 0.2972 - val_acc: 0.8886
Epoch 43/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3178 - acc: 0.8844 - val_loss: 0.2941 - val_acc: 0.8873
Epoch 44/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3170 - acc: 0.8834 - val_loss: 0.2929 - val_acc: 0.8847
Epoch 45/100
3087/3087 [==============================] - 6s 

Epoch 98/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3048 - acc: 0.8892 - val_loss: 0.2888 - val_acc: 0.8886
Epoch 99/100
3087/3087 [==============================] - 15s 5ms/step - loss: 0.3025 - acc: 0.8853 - val_loss: 0.2890 - val_acc: 0.8886
Epoch 100/100
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3038 - acc: 0.8866 - val_loss: 0.2888 - val_acc: 0.8886
Test accuracy: 0.8886010362694301
Train on 3087 samples, validate on 772 samples
Epoch 1/150
3087/3087 [==============================] - 17s 5ms/step - loss: 0.5615 - acc: 0.6991 - val_loss: 0.4074 - val_acc: 0.8653
Epoch 2/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.3901 - acc: 0.8614 - val_loss: 0.3262 - val_acc: 0.8769
Epoch 3/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.3365 - acc: 0.8827 - val_loss: 0.3009 - val_acc: 0.8886
Epoch 4/150
3087/3087 [==============================] - 11s 3ms/step - loss: 0.3254 - acc: 0.8831 

Epoch 56/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2899 - acc: 0.8889 - val_loss: 0.2825 - val_acc: 0.8886
Epoch 57/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2897 - acc: 0.8902 - val_loss: 0.2824 - val_acc: 0.8886
Epoch 58/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2958 - acc: 0.8915 - val_loss: 0.2822 - val_acc: 0.8899
Epoch 59/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2880 - acc: 0.8902 - val_loss: 0.2824 - val_acc: 0.8886
Epoch 60/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2873 - acc: 0.8905 - val_loss: 0.2823 - val_acc: 0.8886
Epoch 61/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2884 - acc: 0.8928 - val_loss: 0.2826 - val_acc: 0.8886
Epoch 62/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2919 - acc: 0.8908 - val_loss: 0.2825 - val_acc: 0.8886
Epoch 63/150
3087/3087 [=================

Epoch 116/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2841 - acc: 0.8937 - val_loss: 0.2828 - val_acc: 0.8886
Epoch 117/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2886 - acc: 0.8912 - val_loss: 0.2829 - val_acc: 0.8886
Epoch 118/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2871 - acc: 0.8915 - val_loss: 0.2828 - val_acc: 0.8886
Epoch 119/150
3087/3087 [==============================] - 14s 4ms/step - loss: 0.2871 - acc: 0.8892 - val_loss: 0.2828 - val_acc: 0.8899
Epoch 120/150
3087/3087 [==============================] - 12s 4ms/step - loss: 0.2849 - acc: 0.8921 - val_loss: 0.2827 - val_acc: 0.8899
Epoch 121/150
3087/3087 [==============================] - 14s 5ms/step - loss: 0.2873 - acc: 0.8912 - val_loss: 0.2827 - val_acc: 0.8899
Epoch 122/150
3087/3087 [==============================] - 15s 5ms/step - loss: 0.2914 - acc: 0.8944 - val_loss: 0.2827 - val_acc: 0.8899
Epoch 123/150
3087/3087 [=========

...

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3251 - acc: 0.8792 - val_loss: 0.3089 - val_acc: 0.8834
Epoch 63/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3307 - acc: 0.8785 - val_loss: 0.3088 - val_acc: 0.8821
Epoch 64/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3235 - acc: 0.8805 - val_loss: 0.3054 - val_acc: 0.8834
Epoch 65/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3266 - acc: 0.8772 - val_loss: 0.3067 - val_acc: 0.8860
Epoch 66/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3247 - acc: 0.8801 - val_loss: 0.3055 - val_acc: 0.8821
Epoch 67/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3219 - acc: 0.8792 - val_loss: 0.3056 - val_acc: 0.8847
Epoch 68/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3230 - acc: 0.8772 - val_loss: 0.3060 - val_acc: 0.8834
Epoch 69/100
3087/3087 [==============================] - 6s 

3087/3087 [==============================] - 6s 2ms/step - loss: 0.2985 - acc: 0.8876 - val_loss: 0.2853 - val_acc: 0.8925
Epoch 22/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2880 - acc: 0.8918 - val_loss: 0.2861 - val_acc: 0.8821
Epoch 23/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2946 - acc: 0.8928 - val_loss: 0.2976 - val_acc: 0.8782
Epoch 24/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2965 - acc: 0.8921 - val_loss: 0.2809 - val_acc: 0.8886
Epoch 25/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2926 - acc: 0.8895 - val_loss: 0.2850 - val_acc: 0.8873

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 26/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2841 - acc: 0.8915 - val_loss: 0.2804 - val_acc: 0.8873
Epoch 27/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2907 - acc: 0.8915 - val_loss: 0.27

3087/3087 [==============================] - 6s 2ms/step - loss: 0.2817 - acc: 0.8941 - val_loss: 0.2783 - val_acc: 0.8990
Epoch 80/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2803 - acc: 0.8941 - val_loss: 0.2781 - val_acc: 0.8990
Epoch 81/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2813 - acc: 0.8967 - val_loss: 0.2782 - val_acc: 0.8990
Epoch 82/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2793 - acc: 0.8973 - val_loss: 0.2782 - val_acc: 0.8990
Epoch 83/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2825 - acc: 0.8912 - val_loss: 0.2782 - val_acc: 0.8990
Epoch 84/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2818 - acc: 0.8937 - val_loss: 0.2781 - val_acc: 0.8990
Epoch 85/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2800 - acc: 0.8954 - val_loss: 0.2781 - val_acc: 0.8990
Epoch 86/200
3087/3087 [==============================] - 6s 

3087/3087 [==============================] - 6s 2ms/step - loss: 0.2841 - acc: 0.8954 - val_loss: 0.2785 - val_acc: 0.8977
Epoch 140/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2836 - acc: 0.8925 - val_loss: 0.2786 - val_acc: 0.8977
Epoch 141/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2819 - acc: 0.8973 - val_loss: 0.2783 - val_acc: 0.8990
Epoch 142/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2843 - acc: 0.8925 - val_loss: 0.2785 - val_acc: 0.8990
Epoch 143/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2799 - acc: 0.8944 - val_loss: 0.2788 - val_acc: 0.8977
Epoch 144/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2834 - acc: 0.8957 - val_loss: 0.2787 - val_acc: 0.8977
Epoch 145/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2791 - acc: 0.8986 - val_loss: 0.2785 - val_acc: 0.8977
Epoch 146/200
3087/3087 [==============================

Epoch 199/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2796 - acc: 0.8950 - val_loss: 0.2794 - val_acc: 0.8964
Epoch 200/200
3087/3087 [==============================] - 6s 2ms/step - loss: 0.2787 - acc: 0.8999 - val_loss: 0.2794 - val_acc: 0.8964
Test accuracy: 0.8963730569948186
Train on 3087 samples, validate on 772 samples
Epoch 1/50
3087/3087 [==============================] - 8s 2ms/step - loss: 0.6217 - acc: 0.6472 - val_loss: 0.5666 - val_acc: 0.7176
Epoch 2/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5432 - acc: 0.7172 - val_loss: 0.4478 - val_acc: 0.8005
Epoch 3/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4257 - acc: 0.8474 - val_loss: 0.3696 - val_acc: 0.8744
Epoch 4/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3741 - acc: 0.8707 - val_loss: 0.3406 - val_acc: 0.8795
Epoch 5/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3511 - acc: 0.8746 - val_loss: 

In [ ]:
0.8297